<a href="https://colab.research.google.com/github/sophiagemanuel/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [9]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [12]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [25]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView('home_sales')

In [26]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
""").show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [27]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms == 3 AND bathrooms == 3
GROUP BY YEAR
ORDER BY YEAR DESC
""").show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [28]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
GROUP BY YEAR
ORDER BY YEAR DESC
""").show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    290242.99|
|2021|    296330.96|
|2020|    292289.09|
|2019|    289859.14|
+----+-------------+



In [29]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  51|    788128.21|
|  54|    798684.82|
|  69|    750537.94|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|  59|     791453.0|
|  85|   1056336.74|
|  52|    733780.26|
|  71|     775651.1|
|  98|   1053739.33|
|  99|   1061201.42|
|  96|   1017815.92|
| 100|    1026669.5|
|  70|    695865.58|
|  61|    746877.59|
|  75|   1114042.94|
|  78|   1080649.37|
|  89|   1107839.15|
|  77|   1076205.56|
+----+-------------+
only showing top 20 rows

--- 0.6996827125549316 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [32]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  51|    788128.21|
|  54|    798684.82|
|  69|    750537.94|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|  59|     791453.0|
|  85|   1056336.74|
|  52|    733780.26|
|  71|     775651.1|
|  98|   1053739.33|
|  99|   1061201.42|
|  96|   1017815.92|
| 100|    1026669.5|
|  70|    695865.58|
|  61|    746877.59|
|  75|   1114042.94|
|  78|   1080649.37|
|  89|   1107839.15|
|  77|   1076205.56|
+----+-------------+
only showing top 20 rows

--- 0.3399391174316406 seconds ---


In [33]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy('date_built').parquet('p_home_sales', mode='overwrite')

In [34]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('p_home_sales')

In [35]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('p_home_sales')

In [36]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  51|    788128.21|
|  54|    798684.82|
|  69|    750537.94|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|  59|     791453.0|
|  85|   1056336.74|
|  52|    733780.26|
|  71|     775651.1|
|  98|   1053739.33|
|  99|   1061201.42|
|  96|   1017815.92|
| 100|    1026669.5|
|  70|    695865.58|
|  61|    746877.59|
|  75|   1114042.94|
|  78|   1080649.37|
|  89|   1107839.15|
|  77|   1076205.56|
+----+-------------+
only showing top 20 rows

--- 0.43149828910827637 seconds ---


In [38]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table p_home_sales')

DataFrame[]

In [39]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('p_home_sales')


False